# Feature Cardinality Analysis

ratings_recent_n.parquet에 등장하는 영화들의 피처별 유니크 값 분석.
- 고카디널리티 피처(배우, 감독 등)가 임베딩 품질에 미치는 영향 파악
- 피처 선택/필터링 기준 결정에 활용

In [1]:
import sys
sys.path.insert(0, "/Users/jisoo/projects/thesis/carte_test")

from config import PROCESSED

In [2]:
import pandas as pd
import numpy as np

# 데이터 로드
ratings = pd.read_parquet(PROCESSED.RATINGS_PARQUET)
catalog = pd.read_parquet(PROCESSED.MOVIE_CATALOG_PARQUET)

print(f"Ratings 레코드 수: {len(ratings):,}")
print(f"Ratings 내 유니크 영화 수: {ratings['movieId'].nunique():,}")
print(f"카탈로그 전체 영화 수: {len(catalog):,}")

Ratings 레코드 수: 13,717,662
Ratings 내 유니크 영화 수: 54,520
카탈로그 전체 영화 수: 86,272


In [3]:
# ratings에 등장하는 영화만 필터링
movie_ids_in_ratings = ratings['movieId'].unique()
catalog_filtered = catalog[catalog['movieId'].isin(movie_ids_in_ratings)].copy()

print(f"Ratings에 등장하면서 카탈로그에 있는 영화 수: {len(catalog_filtered):,}")
print(f"카탈로그에 없는 영화 수: {len(movie_ids_in_ratings) - len(catalog_filtered):,}")

Ratings에 등장하면서 카탈로그에 있는 영화 수: 53,630
카탈로그에 없는 영화 수: 890


## 1. 피처별 유니크 값 개수

In [4]:
def analyze_feature_cardinality(df: pd.DataFrame, exclude_cols: list = None) -> pd.DataFrame:
    """
    피처별 유니크 값 개수, null 비율 등 분석
    """
    if exclude_cols is None:
        exclude_cols = ['movieId', 'tmdbId']
    
    feature_cols = [col for col in df.columns if col not in exclude_cols]
    
    results = []
    for col in feature_cols:
        non_null = df[col].dropna()
        unique_count = non_null.nunique()
        total_count = len(non_null)
        null_count = len(df) - total_count
        
        results.append({
            'feature': col,
            'unique': unique_count,
            'non_null': total_count,
            'null': null_count,
            'null_pct': null_count / len(df) * 100,
            'unique_ratio': unique_count / total_count if total_count > 0 else 0
        })
    
    return pd.DataFrame(results).sort_values('unique', ascending=False)

cardinality_df = analyze_feature_cardinality(catalog_filtered)
cardinality_df.style.format({
    'unique': '{:,}',
    'non_null': '{:,}',
    'null': '{:,}',
    'null_pct': '{:.1f}%',
    'unique_ratio': '{:.2%}'
})

,feature,unique,non_null,null,null_pct,unique_ratio
3,overview,"53,473","53,551",79,0.1%,99.85%
1,original_title,"51,154","53,630",0,0.0%,95.38%
12,actor_3,"31,419","49,903","3,727",6.9%,62.96%
2,tagline,"29,297","29,600","24,030",44.8%,98.98%
11,actor_2,"28,690","50,788","2,842",5.3%,56.49%
14,writer_1,"26,869","48,599","5,031",9.4%,55.29%
10,actor_1,"24,015","52,283","1,347",2.5%,45.93%
13,director_1,"23,066","53,434",196,0.4%,43.17%
4,produced_by_company_1,"19,420","49,011","4,619",8.6%,39.62%
5,produced_by_company_2,"13,998","30,937","22,693",42.3%,45.25%


## 2. 피처 그룹별 요약

In [5]:
# 피처 그룹 정의
FEATURE_GROUPS = {
    'text_unique': ['overview', 'original_title', 'tagline'],
    'person': ['actor_1', 'actor_2', 'actor_3', 'director_1', 'writer_1'],
    'company': ['produced_by_company_1', 'produced_by_company_2'],
    'categorical': ['produced_in_country_1', 'produced_in_country_2', 
                    'spoken_language_1', 'spoken_language_2',
                    'release_year', 'genre_1', 'genre_2', 'genre_3']
}

print("피처 그룹별 평균 유니크 값 개수")
print("=" * 50)
for group_name, features in FEATURE_GROUPS.items():
    group_stats = cardinality_df[cardinality_df['feature'].isin(features)]
    avg_unique = group_stats['unique'].mean()
    avg_ratio = group_stats['unique_ratio'].mean()
    print(f"{group_name:15s} | 평균 유니크: {avg_unique:>8,.0f} | 평균 유니크비율: {avg_ratio:.2%}")

피처 그룹별 평균 유니크 값 개수
text_unique     | 평균 유니크:   44,641 | 평균 유니크비율: 98.07%
person          | 평균 유니크:   26,812 | 평균 유니크비율: 52.77%
company         | 평균 유니크:   16,709 | 평균 유니크비율: 42.44%
categorical     | 평균 유니크:       96 | 평균 유니크비율: 0.44%


## 3. 고카디널리티 피처 빈도 분석 (배우/감독/작가)

In [6]:
def analyze_value_frequency(df: pd.DataFrame, columns: list, top_n: int = 20) -> dict:
    """
    특정 컬럼들의 값 빈도 분석
    - 전체 값들을 합쳐서 빈도 계산
    - 등장 횟수별 분포 반환
    """
    all_values = []
    for col in columns:
        all_values.extend(df[col].dropna().tolist())
    
    value_counts = pd.Series(all_values).value_counts()
    
    # 등장 횟수별 분포
    freq_dist = value_counts.value_counts().sort_index()
    
    return {
        'total_values': len(all_values),
        'unique_values': len(value_counts),
        'top_values': value_counts.head(top_n),
        'frequency_distribution': freq_dist,
        'value_counts': value_counts
    }

# 배우 분석
actor_cols = ['actor_1', 'actor_2', 'actor_3']
actor_stats = analyze_value_frequency(catalog_filtered, actor_cols)

print("[배우 빈도 분석]")
print(f"총 배우 슬롯: {actor_stats['total_values']:,}")
print(f"유니크 배우 수: {actor_stats['unique_values']:,}")
print(f"\nTop 20 배우:")
print(actor_stats['top_values'])

[배우 빈도 분석]
총 배우 슬롯: 152,974
유니크 배우 수: 62,550

Top 20 배우:
Mel Blanc            135
Nicolas Cage          93
Bruce Willis          91
Robert De Niro        90
John Wayne            84
Gérard Depardieu      83
Michael Caine         78
Clarence Nash         77
Akshay Kumar          74
Samuel L. Jackson     72
Tom Hanks             72
Jackie Chan           69
Morgan Freeman        67
Jeff Bridges          67
Liam Neeson           66
Donald Sutherland     65
Willem Dafoe          65
Pinto Colvig          64
Anthony Hopkins       63
Meryl Streep          61
Name: count, dtype: int64


In [7]:
# 감독 분석
director_stats = analyze_value_frequency(catalog_filtered, ['director_1'])

print("[감독 빈도 분석]")
print(f"총 감독 슬롯: {director_stats['total_values']:,}")
print(f"유니크 감독 수: {director_stats['unique_values']:,}")
print(f"\nTop 20 감독:")
print(director_stats['top_values'])

[감독 빈도 분석]
총 감독 슬롯: 53,434
유니크 감독 수: 23,066

Top 20 감독:
Chuck Jones          66
Jean-Luc Godard      54
Alfred Hitchcock     53
Werner Herzog        51
Woody Allen          51
John Ford            48
Charlie Chaplin      48
Michael Curtiz       48
Georges Méliès       46
Takashi Miike        44
Sidney Lumet         43
Ingmar Bergman       42
Friz Freleng         40
Martin Scorsese      40
Clint Eastwood       39
Steven Spielberg     36
Robert Altman        35
Steven Soderbergh    35
Fritz Lang           35
George Cukor         34
Name: count, dtype: int64


In [8]:
# 작가 분석
writer_stats = analyze_value_frequency(catalog_filtered, ['writer_1'])

print("[작가 빈도 분석]")
print(f"총 작가 슬롯: {writer_stats['total_values']:,}")
print(f"유니크 작가 수: {writer_stats['unique_values']:,}")
print(f"\nTop 20 작가:")
print(writer_stats['top_values'])

[작가 빈도 분석]
총 작가 슬롯: 48,599
유니크 작가 수: 26,869

Top 20 작가:
Woody Allen          52
Michael Maltese      51
Jean-Luc Godard      45
Ingmar Bergman       42
Werner Herzog        41
Charlie Chaplin      41
Luc Besson           32
Ben Hecht            32
Wong Jing            32
Ni Kuang             31
Agnès Varda          30
Tyler Perry          28
John Hughes          28
Charles M. Schulz    27
Nunnally Johnson     26
Neil Simon           26
Hong Sang-soo        26
John Sayles          25
Larry Cohen          25
Mark Monroe          25
Name: count, dtype: int64


## 4. 등장 빈도별 분포 (희귀값 비율)

In [9]:
def summarize_rarity(value_counts: pd.Series, thresholds: list = [1, 2, 5, 10, 20]) -> pd.DataFrame:
    """
    등장 횟수 기준 희귀값 비율 계산
    """
    total = len(value_counts)
    results = []
    
    for t in thresholds:
        count = (value_counts <= t).sum()
        results.append({
            'threshold': f'<= {t}회',
            'count': count,
            'pct': count / total * 100
        })
    
    return pd.DataFrame(results)

print("[배우 희귀값 분포]")
print(summarize_rarity(actor_stats['value_counts']).to_string(index=False))
print()

print("[감독 희귀값 분포]")
print(summarize_rarity(director_stats['value_counts']).to_string(index=False))
print()

print("[작가 희귀값 분포]")
print(summarize_rarity(writer_stats['value_counts']).to_string(index=False))

[배우 희귀값 분포]
threshold  count       pct
    <= 1회  42585 68.081535
    <= 2회  50561 80.832934
    <= 5회  57376 91.728217
   <= 10회  60207 96.254197
   <= 20회  61740 98.705036

[감독 희귀값 분포]
threshold  count       pct
    <= 1회  14245 61.757565
    <= 2회  17988 77.984913
    <= 5회  21213 91.966531
   <= 10회  22427 97.229689
   <= 20회  22925 99.388711

[작가 희귀값 분포]
threshold  count       pct
    <= 1회  18443 68.640441
    <= 2회  22557 83.951766
    <= 5회  25701 95.652983
   <= 10회  26632 99.117943
   <= 20회  26836 99.877182


## 5. 제작사 분석

In [10]:
company_cols = ['produced_by_company_1', 'produced_by_company_2']
company_stats = analyze_value_frequency(catalog_filtered, company_cols)

print("[제작사 빈도 분석]")
print(f"총 제작사 슬롯: {company_stats['total_values']:,}")
print(f"유니크 제작사 수: {company_stats['unique_values']:,}")
print(f"\nTop 20 제작사:")
print(company_stats['top_values'])
print()

print("[제작사 희귀값 분포]")
print(summarize_rarity(company_stats['value_counts']).to_string(index=False))

[제작사 빈도 분석]
총 제작사 슬롯: 79,948
유니크 제작사 수: 27,306

Top 20 제작사:
Warner Bros. Pictures      863
Paramount Pictures         861
Universal Pictures         835
Metro-Goldwyn-Mayer        758
Columbia Pictures          701
20th Century Fox           666
United Artists             311
Walt Disney Productions    309
New Line Cinema            283
TOHO                       241
Mosfilm                    239
Walt Disney Pictures       228
RKO Radio Pictures         224
BBC                        223
BBC Film                   193
Touchstone Pictures        192
Gaumont                    173
StudioCanal                164
Miramax                    160
CJ Entertainment           151
Name: count, dtype: int64

[제작사 희귀값 분포]
threshold  count       pct
    <= 1회  17951 65.740130
    <= 2회  21700 79.469714
    <= 5회  24997 91.543983
   <= 10회  26289 96.275544
   <= 20회  26891 98.480188


## 6. 저카디널리티 피처 분포 (장르, 국가, 언어)

In [11]:
# 장르 분포
genre_cols = ['genre_1', 'genre_2', 'genre_3']
genre_stats = analyze_value_frequency(catalog_filtered, genre_cols, top_n=30)

print("[장르 분포] (전체 장르 값)")
print(genre_stats['top_values'])

[장르 분포] (전체 장르 값)
Drama              23395
Comedy             16937
Thriller            8250
Romance             8106
Action              6968
Horror              6220
Documentary         5719
Crime               5548
Adventure           4158
Science Fiction     3430
Family              3298
Animation           3224
Mystery             3129
Fantasy             2987
Music               2201
History             2140
TV Movie            2082
War                 1487
Western              852
Name: count, dtype: int64


In [12]:
# 국가 분포 (Top 20)
country_cols = ['produced_in_country_1', 'produced_in_country_2']
country_stats = analyze_value_frequency(catalog_filtered, country_cols)

print("[제작 국가 분포] (Top 20)")
print(country_stats['top_values'])

[제작 국가 분포] (Top 20)
United States of America    25683
United Kingdom               5124
France                       4931
Canada                       2669
Germany                      2507
Japan                        2147
Italy                        2072
India                        1772
Spain                        1129
Hong Kong                     846
South Korea                   811
Belgium                       783
Australia                     777
Russia                        764
China                         650
Soviet Union                  635
Sweden                        613
Finland                       545
Denmark                       539
Brazil                        524
Name: count, dtype: int64


In [13]:
# 언어 분포 (Top 20)
lang_cols = ['spoken_language_1', 'spoken_language_2']
lang_stats = analyze_value_frequency(catalog_filtered, lang_cols)

print("[언어 분포] (Top 20)")
print(lang_stats['top_values'])

[언어 분포] (Top 20)
English        34244
French          4768
Spanish         2966
German          2586
Italian         2295
Japanese        2246
Russian         1780
Mandarin        1064
Hindi           1047
No Language     1039
Portuguese       843
Korean           771
Cantonese        605
Arabic           518
Swedish          516
Polish           478
Finnish          462
Turkish          379
Danish           357
Dutch            329
Name: count, dtype: int64


In [14]:
# 연도 분포
year_counts = catalog_filtered['release_year'].value_counts().sort_index()

print("[개봉 연도 분포]")
print(f"연도 범위: {year_counts.index.min()} ~ {year_counts.index.max()}")
print(f"\n최근 10년 영화 수:")
print(year_counts.tail(10))

[개봉 연도 분포]
연도 범위: 1874 ~ 2025

최근 10년 영화 수:
release_year
2016    2020
2017    2158
2018    2166
2019    2075
2020    1703
2021    1685
2022    1726
2023     942
2024      31
2025      11
Name: count, dtype: Int64


## 7. 요약 및 권장사항

In [ ]:
print("=" * 70)
print("피처 카디널리티 분석 요약")
print("=" * 70)
print()

# 희귀값 비율 (1회만 등장)
actor_rare = (actor_stats['value_counts'] == 1).sum() / len(actor_stats['value_counts']) * 100
director_rare = (director_stats['value_counts'] == 1).sum() / len(director_stats['value_counts']) * 100
writer_rare = (writer_stats['value_counts'] == 1).sum() / len(writer_stats['value_counts']) * 100
company_rare = (company_stats['value_counts'] == 1).sum() / len(company_stats['value_counts']) * 100

print(f"{'피처':<20} {'유니크값':<12} {'1회등장비율':<15}")
print("-" * 50)
print(f"{'배우 (actor_1~3)':<20} {actor_stats['unique_values']:<12,} {actor_rare:.1f}%")
print(f"{'감독 (director_1)':<20} {director_stats['unique_values']:<12,} {director_rare:.1f}%")
print(f"{'작가 (writer_1)':<20} {writer_stats['unique_values']:<12,} {writer_rare:.1f}%")
print(f"{'제작사 (company_1~2)':<20} {company_stats['unique_values']:<12,} {company_rare:.1f}%")
print(f"{'장르 (genre_1~3)':<20} {genre_stats['unique_values']:<12,} -")
print(f"{'국가 (country_1~2)':<20} {country_stats['unique_values']:<12,} -")
print(f"{'언어 (language_1~2)':<20} {lang_stats['unique_values']:<12,} -")
print()

피처 카디널리티 분석 요약

피처                   유니크값         1회등장비율         
--------------------------------------------------
배우 (actor_1~3)       62,550       68.1%
감독 (director_1)      23,066       61.8%
작가 (writer_1)        26,869       68.6%
제작사 (company_1~2)    27,306       65.7%
장르 (genre_1~3)       19           -
국가 (country_1~2)     178          -
언어 (language_1~2)    151          -

[권장사항]
- 배우/감독/작가: 1회만 등장하는 값이 50% 이상이면 피처 효과 제한적
- 고려 방안:
  1) 최소 등장 횟수 필터링 (예: 5회 이상만 유지, 나머지는 'OTHER')
  2) 인물 피처 제외하고 장르/국가/연도만 사용
  3) 인물별 별도 임베딩 학습 후 lookup
